In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:,3:13].values # Taking only relavent inputs
Y = dataset.iloc[:,13].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#handling categorical data like geography and gender
ct = ColumnTransformer([("Geography", OneHotEncoder(),[1])], remainder="passthrough") # The last arg ([0]) is the list of columns you want to transform in this step
X = ct.fit_transform(X)
X = X[:,1:]
ct_2 = ColumnTransformer([("Gender", OneHotEncoder(),[3])], remainder="passthrough") # The last arg ([0]) is the list of columns you want to transform in this step
X = ct_2.fit_transform(X)
X = X[:,1:]
X[0:1,:]

array([[0.0, 0.0, 0.0, 619, 42, 2, 0.0, 1, 1, 1, 101348.88]], dtype=object)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

In [7]:
# Now in XGBoost we dont need scale our data it manage it. it keeps our data in original form
# it is actually gradient boosting algorithm with decision trees

In [9]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [10]:
y_pred = classifier.predict(X_test) # Here we will get our answer in 0 or 1 no probability

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

array([[1521,   74],
       [ 197,  208]], dtype=int64)

In [11]:
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = classifier, X = X_train, y = Y_train, cv = 10, n_jobs = -1)
    # n_jobs = -1 means to use all CPU
accuracies

array([0.87   , 0.855  , 0.87875, 0.8725 , 0.86   , 0.8525 , 0.865  ,
       0.85   , 0.84875, 0.8725 ])

In [12]:
accuracies.mean()

0.8625

In [14]:
accuracies.std()

0.01017042280340401